In [46]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

ds = pd.read_csv("perfect_datasets/perfect_dataset_v3.csv")
ds_overview = pd.read_csv("perfect_datasets/perfect_dataset_v1.csv")

cv = CountVectorizer(max_features=5000, stop_words="english")

vectors = cv.fit_transform(ds["tags"]).toarray()

# cv.get_feature_names_out().tolist()

similarity = cosine_similarity(vectors)

def recommend(movie):
    index = ds[ds["title"].str.lower() == movie.lower()].index[0]
    distance = similarity[index]
    movie = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:6]

    for i in movie:
        print("Movie Id: " + str(ds.iloc[i[0]].movie_id))
        print("Movie Name: " + ds.iloc[i[0]].title)
        print("Movie Name: " + ds_overview.iloc[i[0]].overview, end="\n\n")

recommend("x-men: apocalypse")

Movie Id: 49538
Movie Name: X-Men: First Class
Movie Name: Before Charles Xavier and Erik Lensherr took the names Professor X and Magneto, they were two young men discovering their powers for the first time. Before they were arch-enemies, they were closest of friends, working together with other mutants (some familiar, some new), to stop the greatest threat the world has ever known.

Movie Id: 127585
Movie Name: X-Men: Days of Future Past
Movie Name: The ultimate X-Men ensemble fights a war for the survival of the species across two time periods as they join forces with their younger selves in an epic battle that must change the past – to save our future.

Movie Id: 36657
Movie Name: X-Men
Movie Name: Two mutants, Rogue and Wolverine, come to a private academy for their kind whose resident superhero team, the X-Men, must oppose a terrorist organization with similar powers.

Movie Id: 76170
Movie Name: The Wolverine
Movie Name: Wolverine faces his ultimate nemesis - and tests of his phy